In [110]:
# Importing dependencies for comunicating with sqlite db
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine,inspect, func, desc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pyodbc
import sqlalchemy as sa
import urllib
import pandas as pd
import os

In [111]:
# Create engine to connect to Sensors.db
engine = create_engine('sqlite:///data/Sensors.db', echo=False)
Base = automap_base()

In [112]:
# Using Base.prepare and Base.classes we will reflect DB into ORM classes
Base.prepare(engine, reflect=True)
Base.classes.keys()

['BME_DATA']

In [113]:
# Explore database with inspector
inspector = inspect(engine)
inspector.get_table_names()

['BME_DATA']

In [114]:
# Save the refference from table BME_DATA
Rambo = Base.classes.BME_DATA

In [115]:
# Create db sesion object
session = Session(engine)

In [116]:
# Creating the query to identify last date to identify end ponint of the year
df = pd.read_sql("SELECT * FROM BME_DATA", engine)

In [117]:
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'])

In [118]:
# using funciton to convert string into date time
from datetime import datetime, timedelta

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))


In [119]:
# testing on  single value
test = hour_rounder(df['TIME_STAMP'][0])

In [120]:
# converting fro all values in entire column
df['TIME_STAMP'] = df['TIME_STAMP'].apply(hour_rounder)

In [121]:
# converting temperature from C to f
df['TEMPERATURE'] = round((df['TEMPERATURE']* 9/5) + 32)


In [122]:
# create new tale and drop Id
df = df[["TIME_STAMP", "TEMPERATURE"]]

In [123]:
df.head()

,TIME_STAMP,TEMPERATURE
0,2019-04-09 23:00:00,167.0
1,2019-04-09 23:00:00,68.0
2,2019-04-10 00:00:00,68.0
3,2019-04-10 01:00:00,67.0
4,2019-04-10 02:00:00,67.0


In [124]:
# Deleting the wrong input at the first row
df = df.iloc[1:]
df.head()

,TIME_STAMP,TEMPERATURE
1,2019-04-09 23:00:00,68.0
2,2019-04-10 00:00:00,68.0
3,2019-04-10 01:00:00,67.0
4,2019-04-10 02:00:00,67.0
5,2019-04-10 03:00:00,66.0


In [125]:
# Importing API data
API_df = pd.read_csv("data/API.csv")
API_df.head()

,City,Hour,F_Temp,A_Temp
0,Boston,4/11/19 3:00,36.61,36.28
1,Boston,4/11/19 4:00,35.21,35.48
2,Boston,4/11/19 5:00,33.81,34.90
3,Boston,4/11/19 6:00,32.91,34.85
4,Boston,4/11/19 7:00,33.05,34.46


In [ ]:
# Merging API and RAMBO dataframes On date
#master_df = pd.merge(fifa_subset, gdp_subset, on = "Nationality", how = "left", suffixes=("_",""))

In [100]:
# Export to csv
df.to_csv("Rambo.csv", sep=',', encoding='utf-8')
df.head()

,TIME_STAMP,TEMPERATURE
1,2019-04-09 23:00:00,68.0
2,2019-04-10 00:00:00,68.0
3,2019-04-10 01:00:00,67.0
4,2019-04-10 02:00:00,67.0
5,2019-04-10 03:00:00,66.0


In [130]:
# Importing API data
dfr = pd.read_csv("data/Rambo.csv")
dfr.head()

,TIME_STAMP,TEMPERATURE,F_TEMP,A_TEMP
0,2019-04-11 3:00,70,65.45,65.81
1,2019-04-11 4:00,68,62.79,64.10
2,2019-04-11 5:00,67,60.54,63.04
3,2019-04-11 6:00,67,60.48,62.80
4,2019-04-11 7:00,67,60.08,62.19


In [131]:
# Convert back to string for Json 
dfr['TIME_STAMP'] = dfr['TIME_STAMP'].dt.strftime('%m-%d-%Y %H:%M')
dfr.head()

AttributeError: Can only use .dt accessor with datetimelike values

In [134]:
df_json_ready = dfr.to_json(orient='records')

In [135]:
df_json_ready

'[{"TIME_STAMP":"2019-04-11 3:00","TEMPERATURE":70,"F_TEMP":65.45,"A_TEMP":65.81},{"TIME_STAMP":"2019-04-11 4:00","TEMPERATURE":68,"F_TEMP":62.79,"A_TEMP":64.1},{"TIME_STAMP":"2019-04-11 5:00","TEMPERATURE":67,"F_TEMP":60.54,"A_TEMP":63.04},{"TIME_STAMP":"2019-04-11 6:00","TEMPERATURE":67,"F_TEMP":60.48,"A_TEMP":62.8},{"TIME_STAMP":"2019-04-11 7:00","TEMPERATURE":67,"F_TEMP":60.08,"A_TEMP":62.19},{"TIME_STAMP":"2019-04-11 8:00","TEMPERATURE":67,"F_TEMP":60.08,"A_TEMP":61.62},{"TIME_STAMP":"2019-04-11 9:00","TEMPERATURE":67,"F_TEMP":60.88,"A_TEMP":60.08},{"TIME_STAMP":"2019-04-11 10:00","TEMPERATURE":69,"F_TEMP":60.65,"A_TEMP":58.9},{"TIME_STAMP":"2019-04-11 11:00","TEMPERATURE":70,"F_TEMP":60.1,"A_TEMP":57.95},{"TIME_STAMP":"2019-04-11 12:00","TEMPERATURE":72,"F_TEMP":60.14,"A_TEMP":58.52},{"TIME_STAMP":"2019-04-11 13:00","TEMPERATURE":74,"F_TEMP":63.5,"A_TEMP":61.8},{"TIME_STAMP":"2019-04-11 14:00","TEMPERATURE":75,"F_TEMP":67.9,"A_TEMP":65.99},{"TIME_STAMP":"2019-04-11 15:00","TEMPER

In [21]:
df.to_json(r'Rambo.json')